# Block Sort Based Indexing

In [1]:
import os

In [8]:
class BSBI:

    def __init__(self):
        self.termId = {}
        self.numTerms = 0

    def parseBlock(self, docsBlock, blockNum):
        terms = []
        for i in range(len(docsBlock)):
            text = docsBlock[i]
            terms.extend(text.split())

        terms = sorted(list(set(terms)))
        for term in terms:
            if term not in self.termId:
                self.numTerms += 1
                self.termId[term] = self.numTerms

        index = {}
        
        for i in range(len(terms)):
            for j in range(len(docsBlock)):
                if terms[i] in docsBlock[j]:
                    if terms[i] in index:
                        index[terms[i]].append(j+1)
                    else:
                        index[terms[i]] = [j+1]
        index = dict(sorted(index.items()))
        strs = []
        with open(f"Sample/doc{blockNum}.csv", "w") as f:
            for k, v in index.items():
                for doc in v:
                    strs.append(f"{self.termId[k]}, {doc}\n")
            strs[-1] = strs[-1].replace('\n','')
            f.writelines(strs)

    def bsbiInvert(self, numBlocks):
        blockSize = 50 // numBlocks
        files = os.listdir('cranfieldDocs')
        for i in range(numBlocks):
            docs = []
            for j in range(blockSize):
                with open(f"cranfieldDocs/{files[i*blockSize + j]}") as f:
                    docs.append(f.read())
            self.parseBlock(docs,i)
        
    def buildIndex(self, numBlocks):
        self.bsbiInvert(numBlocks)

        index = {}

        for i in range(numBlocks):
            with open(f"Sample/doc{i}.csv", "r") as f:
                text = f.read()
                data = [[int(x) for x in l.split(', ')] for l in text.split('\n')]
                for d in data:
                    val = list(self.termId.keys())[list(self.termId.values()).index(d[0])]
                    if val not in index:
                        index[val] = [d[1]]
                    else:
                        index[val].append(d[1])
                    index[val] = sorted(list(set(index[val])))
        myKeys = list(index.keys())
        myKeys.sort()
        sortedIndex = {i: index[i] for i in myKeys}
        with open(f"Sample/Merged.csv", "w") as f:
            for k, v in sortedIndex.items():
                for doc in v:
                    s = f"{self.termId[k]}, {doc}\n"
                    f.write(s)

In [9]:
bsbi = BSBI()

In [10]:
bsbi.buildIndex(10)

FileNotFoundError: [Errno 2] No such file or directory: 'Sample/doc0.csv'